In [7]:
import pandas as pd

**Load datasets**

In [55]:
name_basics = pd.read_csv("name.basics.tsv", sep="\t", low_memory=False, na_values="\\N")
title_basics = pd.read_csv("title.basics.tsv", sep="\t", low_memory=False, na_values="\\N")
title_ratings = pd.read_csv("title.ratings.tsv", sep="\t", low_memory=False, na_values="\\N")

In [56]:
print(name_basics.columns)

Index(['nconst', 'primaryName', 'birthYear', 'deathYear', 'primaryProfession',
       'knownForTitles'],
      dtype='object')


In [59]:
name_basics.columns = name_basics.columns.str.strip()

**Cleaning the data name.basics**

Just getting the usefull columns

In [61]:
name_basics = name_basics[["nconst", "primaryName", "knownForTitles"]]
name_basics.head()

,nconst,primaryName,knownForTitles
0,nm0000001,Fred Astaire,"tt0072308,tt0050419,tt0027125,tt0031983"
1,nm0000002,Lauren Bacall,"tt0037382,tt0075213,tt0038355,tt0117057"
2,nm0000003,Brigitte Bardot,"tt0057345,tt0049189,tt0056404,tt0054452"
3,nm0000004,John Belushi,"tt0072562,tt0077975,tt0080455,tt0078723"
4,nm0000005,Ingmar Bergman,"tt0050986,tt0069467,tt0050976,tt0083922"


**Cleaning the data  title.basics**

In [63]:
title_basics = title_basics[["tconst", "primaryTitle", "genres"]]
title_basics.head()

,tconst,primaryTitle,genres
0,tt0000001,Carmencita,"Documentary,Short"
1,tt0000002,Le clown et ses chiens,"Animation,Short"
2,tt0000003,Poor Pierrot,"Animation,Comedy,Romance"
3,tt0000004,Un bon bock,"Animation,Short"
4,tt0000005,Blacksmith Scene,Short


**Cleaning the data title.ratings**

In [65]:
title_ratings = title_ratings[["tconst", "averageRating"]]
title_ratings.head()

,tconst,averageRating
0,tt0000001,5.7
1,tt0000002,5.5
2,tt0000003,6.4
3,tt0000004,5.3
4,tt0000005,6.2


**Merging Datasets title_basics with title_ratings**

In [67]:
movies_df = pd.merge(title_basics, title_ratings, on="tconst", how="left")
movies_df.head()

,tconst,primaryTitle,genres,averageRating
0,tt0000001,Carmencita,"Documentary,Short",5.7
1,tt0000002,Le clown et ses chiens,"Animation,Short",5.5
2,tt0000003,Poor Pierrot,"Animation,Comedy,Romance",6.4
3,tt0000004,Un bon bock,"Animation,Short",5.3
4,tt0000005,Blacksmith Scene,Short,6.2


**Merging with name_basics (Actors and Directors)**

Explode knownForTitles column to match tconst

Merge actors with movies

Drop knownForTitles column (as we already merged it)

In [69]:
name_basics["knownForTitles"] = name_basics["knownForTitles"].fillna("")
name_basics_expanded = name_basics.assign(knownForTitles=name_basics["knownForTitles"].str.split(","))
name_basics_expanded = name_basics_expanded.explode("knownForTitles")

final_df = pd.merge(movies_df, name_basics_expanded, left_on="tconst", right_on="knownForTitles", how="left")

final_df.drop(columns=["knownForTitles"], inplace=True)

final_df.head()

,tconst,primaryTitle,genres,averageRating,nconst,primaryName
0,tt0000001,Carmencita,"Documentary,Short",5.7,nm1588970,Carmencita
1,tt0000002,Le clown et ses chiens,"Animation,Short",5.5,NaN,NaN
2,tt0000003,Poor Pierrot,"Animation,Comedy,Romance",6.4,nm0721526,Émile Reynaud
3,tt0000003,Poor Pierrot,"Animation,Comedy,Romance",6.4,nm1335271,Gaston Paulin
4,tt0000003,Poor Pierrot,"Animation,Comedy,Romance",6.4,nm17045636,Louis Morin


**Saving dataset merged

In [71]:
final_df.to_csv("merged_imdb_data.csv", index=False)